# Module 04: Human-in-the-Loop - Follow Along

**Key Topics:** InjectedState, Tools with Command, Breakpoints, Approval workflows

Run all HITL examples from the theory document.


In [ ]:
%pip install -q -U langgraph langchain

from langgraph.prebuilt import InjectedState
from langgraph.types import Command
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
from langchain.tools import tool

print('✅ HITL tools ready!')

## Example 1: InjectedState for Context-Aware Tools

Tools can access state without explicit parameters.


In [ ]:
class AgentState(TypedDict):
    user_id: str
    query: str
    result: str

@tool
def search_with_context(
    query: str,
    state: Annotated[dict, InjectedState]
) -> str:
    """Search tool that knows the user context."""
    user_id = state.get('user_id', 'unknown')
    return f"Search results for '{query}' (user: {user_id}): Result 1, Result 2"

# Test the tool
test_state = {'user_id': 'user123', 'query': 'LangGraph', 'result': ''}
result = search_with_context.invoke(
    {'query': 'LangGraph', 'state': test_state}
)
print(result)
print('✅ Tool accessed state without explicit parameter!')

## Example 2: Tool Returns Command

Tools can return Commands to update state and control flow.


In [ ]:
@tool
def risky_action(
    action: str,
    state: Annotated[dict, InjectedState]
) -> Command:
    """Tool that requires approval for risky actions."""
    if 'delete' in action.lower() or 'remove' in action.lower():
        return Command(
            update={'result': f'APPROVAL NEEDED: {action}'},
            goto='approval_required'
        )
    return Command(
        update={'result': f'Executed: {action}'},
        goto='complete'
    )

# Simulate tool call
safe_result = risky_action.invoke({'action': 'read file', 'state': {}})
risky_result = risky_action.invoke({'action': 'delete file', 'state': {}})

print(f"Safe action: {safe_result.update}")
print(f"Risky action: {risky_result.update}")
print('✅ Tools can control routing with Command!')

## Example 3: Breakpoints for Human Approval


In [ ]:
class ApprovalState(TypedDict):
    action: str
    approved: bool
    result: str

def plan_action(state: ApprovalState):
    return {'action': 'Delete important file'}

def execute_action(state: ApprovalState):
    if state.get('approved'):
        return {'result': f"Executed: {state['action']}"}
    return {'result': 'Action cancelled'}

# Build graph with breakpoint
workflow = StateGraph(ApprovalState)
workflow.add_node('plan', plan_action)
workflow.add_node('execute', execute_action)
workflow.add_edge(START, 'plan')
workflow.add_edge('plan', 'execute')
workflow.add_edge('execute', END)

# Compile with checkpoint and interrupt
memory = MemorySaver()
app = workflow.compile(
    checkpointer=memory,
    interrupt_before=['execute']  # Pause before execute
)

# Run first part
config = {'configurable': {'thread_id': 'demo'}}
result = app.invoke({'action': '', 'approved': False, 'result': ''}, config)
print(f"Planned action: {result['action']}")
print('⏸️  Execution paused - waiting for approval')

# Resume with approval
result = app.invoke({'approved': True}, config)
print(f"Result: {result['result']}")
print('✅ Human-in-the-loop workflow complete!')

## Example 4: Multi-Stage Approval


In [ ]:
class MultiApprovalState(TypedDict):
    plan: str
    budget_approved: bool
    security_approved: bool
    final_result: str

def create_plan(state):
    return {'plan': 'Deploy to production'}

def check_budget(state):
    # In real app: wait for budget approval
    return {'budget_approved': state.get('budget_approved', False)}

def check_security(state):
    # In real app: wait for security approval  
    return {'security_approved': state.get('security_approved', False)}

def execute_deployment(state):
    if state['budget_approved'] and state['security_approved']:
        return {'final_result': 'DEPLOYED ✅'}
    return {'final_result': 'BLOCKED ❌'}

# Build multi-stage approval
workflow = StateGraph(MultiApprovalState)
workflow.add_node('plan', create_plan)
workflow.add_node('budget', check_budget)
workflow.add_node('security', check_security)
workflow.add_node('deploy', execute_deployment)

workflow.add_edge(START, 'plan')
workflow.add_edge('plan', 'budget')
workflow.add_edge('budget', 'security')
workflow.add_edge('security', 'deploy')
workflow.add_edge('deploy', END)

memory = MemorySaver()
app = workflow.compile(
    checkpointer=memory,
    interrupt_before=['budget', 'security']  # Two approval gates
)

print('Multi-stage approval workflow created')
print('Requires budget AND security approval before deployment')

## Summary

You've run examples for:
- ✅ InjectedState for context-aware tools
- ✅ Tools returning Command for routing
- ✅ Breakpoints for human approval
- ✅ Multi-stage approval workflows

**Next:** `module-04-practice.ipynb` for HITL exercises!
